## XTZ deposit to contract with `receive()` test

In [1]:
from docs.scenarios.setup import *

accounts = setup()
web3, etherlink_account, tezos_account = accounts

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `100.376908 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `1.661226239 ꜩ`


### Deploy DepositTester:
- This is the contract that have `receive()` entrypoint which redirects incoming `value` to the `xtz_withdrawal_precompile`.
- First it is checked that this entrypoint was not executed during native bridge xtz deposit.
- Then it is checked that this entrypoint was executed during L2 transfer with `value` to the `DepositTester`.

In [2]:
deposit_helper = DepositTesterHelper.originate_from_file(
    web3=web3,
    account=etherlink_account,
    filename=make_filename('DepositTester'),
    constructor_args=(
        XTZ_WITHDRAWAL_PRECOMPILE,
        get_address(tezos_account),
    )
)
deposit_helper.address

'0xCef9Fd72C44B0f13e80c799B5530DC58b4B1789c'

### Native deposit to the DepositTester:

In [3]:
opg_hash = xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    amount=1,
    receiver_address=deposit_helper.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL
)

Making XTZ deposit using Helper `KT1MJxf4KVN3sosR99VRG7WBbWTJtAyWUJt9`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA`
      * Receiver address: `0xCef9Fd72C44B0f13e80c799B5530DC58b4B1789c`
      * Amount (mutez): `1`
Successfully executed XTZ deposit, tx hash: `oo63cygDLJpHHRJupGf8VEiMe382EGoDpdKZnZztqVA8HJ9nDGR`


- TODO: Add a check that first transaction (called by Kernel during deposit) doesn't created any logs
- TODO: Add a check that `DepositTester` balance increased

### XTZ transfer to the DepositTester:

In [4]:
tx_hash = etherlink_legacy_transfer(web3, etherlink_account, deposit_helper, 10**12)

Sending Ethernlink xtz:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Receiver: `0xCef9Fd72C44B0f13e80c799B5530DC58b4B1789c`
  - Amount (wei): `1_000_000_000_000`
  - Amount (mutez): `1`
Successfully transfered, tx hash: `0x796a5ee93bd14d975792b5a204cdc0e30f04c2ad3728fb3fc1d7ee9dfb590a4e`


HexBytes('0x796a5ee93bd14d975792b5a204cdc0e30f04c2ad3728fb3fc1d7ee9dfb590a4e')

- check that second transaction (called by etherlink_account) created logs with xtz withdawal
- check that `DepositTester` balance wasn't changed